#### Import Necessary Library

In [1]:
import os
from dotenv import load_dotenv

from langchain_groq import ChatGroq   # Or ChatOpenAI if you prefer OpenAI
from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.documents import Document
import streamlit as st
from langchain_core.prompts import ChatPromptTemplate
#from langchain_core.output_parsers import StrOutputParser
#from langchain_community.llms import Ollama

In [ ]:
## Langsmith Tracking
#os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
#os.environ["LANGCHAIN_TRACING_V2"]="true"
#os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
# set embedding tech
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

g:\Gen AI-Krish Naik\5. Personal Gen AI Project\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [4]:
# load the data and fetch the index path
PDF_PATH = "Customer.pdf"
INDEX_PATH = "faiss_index"

In [5]:
from langchain_community.vectorstores import FAISS
# load from disk
new_db = FAISS.load_local(INDEX_PATH,embeddings,allow_dangerous_deserialization=True)

In [6]:
query="customer_query  intent  priority"
docs=new_db.similarity_search(query)
docs

[Document(metadata={'source': 'Customer.pdf', 'page': 0}, page_content='ticket_id  \nchannel  \ncustomer_query  intent  priority  \nagent_response  \n1001  \nSocial  \nLaptop  wonâ€™t  start  even  after  charging.  \ntier1_triage  \nHigh  \nGuide  troubleshooting,  create  ticket,  escalate  to  L2.  \n1002  \nSocial  \nThe  app  keeps  crashing  when  I  try  to  log  in.  \ntechnical_issue  \nHigh  \nGather  logs,  create  ticket,  escalate  to  technical  team.  \n1003  \nWhatsApp  \nMy  router  light  is  blinking  red.  tier1_triage  \nMedium  \nGuide  through  restart  steps,  escalate  if  unresolved.  \n1004  \nChatbot  \nCan  I  change  my  delivery  address  before  shipping?  \norder_modification  \nHigh  \nConfirm  order  status,  update  address,  send  confirmation.  \n1005  \nPhone  \nMy  payment  was  deducted  twice,  what  should  I  do?  \nrefund_request  \nMedium  \nAcknowledge,  verify,  initiate  refund,  confirm  turnaround  time.  \n1006  \nSocial  \nWe  notice

In [9]:
print(docs[0].page_content)

ticket_id  
channel  
customer_query  intent  priority  
agent_response  
1001  
Social  
Laptop  wonâ€™t  start  even  after  charging.  
tier1_triage  
High  
Guide  troubleshooting,  create  ticket,  escalate  to  L2.  
1002  
Social  
The  app  keeps  crashing  when  I  try  to  log  in.  
technical_issue  
High  
Gather  logs,  create  ticket,  escalate  to  technical  team.  
1003  
WhatsApp  
My  router  light  is  blinking  red.  tier1_triage  
Medium  
Guide  through  restart  steps,  escalate  if  unresolved.  
1004  
Chatbot  
Can  I  change  my  delivery  address  before  shipping?  
order_modification  
High  
Confirm  order  status,  update  address,  send  confirmation.  
1005  
Phone  
My  payment  was  deducted  twice,  what  should  I  do?  
refund_request  
Medium  
Acknowledge,  verify,  initiate  refund,  confirm  turnaround  time.  
1006  
Social  
We  noticed  you  havenâ€™t  logged  in  recently,  need  help?  
proactive_outreach  
Low


In [7]:
# retriever object to query similar documents
retriever = new_db.as_retriever(search_type="similarity", search_kwargs={"k": 3})
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001C43624A900>, search_kwargs={'k': 3})

In [ ]:
# %% 2. Define LLM

In [10]:
import os
from dotenv import load_dotenv
import streamlit as st
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [11]:
load_dotenv()
## load the GROQ API Key
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")
os.environ['GROQ_API_KEY']=os.getenv("GROQ_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.1-8b-instant")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001C4364FB8C0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001C425E2EDB0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [30]:
## Prompt Template
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [31]:
## Document chain for creation-custom prompt for answering questions with context.

document_chains =  create_stuff_documents_chain(llm,prompt)
document_chains

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise.\n\n{context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001C4364FB8C0>, async_client=<groq.resources.chat.completions.AsyncCompletio

In [32]:
from langchain_core.documents import Document
document_chains.invoke({
    "input":"The  app  keeps  crashing  when  I  try  to  log  in.",
    "context":[Document(page_content="The  app  keeps  crashing  when  I  try  to  log  in. technical_issue High Gather  logs,  create  ticket,  escalate  to  technical  team. 1003 WhatsApp  My  router  light  is  blinking  red.  tier1_triage. ")]
})

'The app keeps crashing when you try to log in. I would suggest gathering logs and creating a ticket to escalate to the technical team. This will help them diagnose and resolve the issue.'

In [14]:
### Input--->Retriever--->vectorstoredb
new_db

In [35]:
# %% 3. Conversational Retriever Setup
retriever = new_db.as_retriever()

from langchain.chains import create_retrieval_chain

Rag_chains = create_retrieval_chain(retriever,document_chains)


In [36]:
Rag_chains

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001C43624A900>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't 

In [37]:
## Get the response form the LLM
response=Rag_chains.invoke({"input":"The  app  keeps  crashing  when  I  try  to  log  in."})
response['answer']

'Gather logs, create ticket, escalate to technical team.'

### User and AI Chat History


In [38]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# Prompt template for conversation
qa_user_prompts = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Use the context to answer the question."),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])

In [39]:
# History-aware retriever (rephrases user queries based on conversation)
history_aware_retriever = create_history_aware_retriever( llm, retriever, qa_user_prompts )
history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001C43624A900>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='A

In [40]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [41]:
# Full retrieval chain (retriever + doc_chain)
new_document_chain = create_stuff_documents_chain(llm,qa_prompt)
new_rag_chain = create_retrieval_chain(history_aware_retriever,new_document_chain)
new_rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001C43624A900>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], ty

#### chat conversation history store

In [54]:
# manual chat history store
from langchain_core.messages import AIMessage,HumanMessage
chat_history=[]
question="Where is my recent order?"
response1=new_rag_chain.invoke({"input":question,"chat_history":chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response1["answer"])
    ]
)

question2="why My router light is blinking red."
response2=new_rag_chain.invoke({"input":question,"chat_history":chat_history})
print(response2['answer'])

I see you've asked the same question before. Check shipment status, provide update, escalate if pending.


In [55]:
chat_history

[HumanMessage(content='Where is my recent order?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Check shipment status, provide update, escalate if pending.', additional_kwargs={}, response_metadata={})]

In [68]:
# automatically store all chat history 
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Wrap your RAG chain with history support
conversational_rag_chain = RunnableWithMessageHistory(
    new_rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [70]:
result = conversational_rag_chain.invoke(
        {"input": "Can I change my delivery  address before shipping?"},
        config={"configurable": {"session_id": "user1"}}
    )

In [71]:
result

{'input': 'Can I change my delivery  address before shipping?',
 'chat_history': [],
 'context': [Document(metadata={'source': 'Customer.pdf', 'page': 7}, page_content='Low  \nCheck  order  status,  cancel  if  possible,  confirm  cancellation.  \n1092  \nWhatsApp  \nCan  I  change  my  delivery  address  before  shipping?  \norder_modification  \nHigh  \nConfirm  order  status,  update  address,  send  confirmation.'),
  Document(metadata={'source': 'Customer.pdf', 'page': 4}, page_content='knowledge_lookup  \nLow  \nRetrieve  latest  HR  policy  and  attach.  \n1057  \nWhatsApp  \nCan  I  change  my  delivery  address  before  shipping?  \norder_modification  \nLow  \nConfirm  order  status,  update  address,  send  confirmation.'),
  Document(metadata={'source': 'Customer.pdf', 'page': 8}, page_content='1098  \nEmail  \nWe  noticed  you  havenâ€™t  logged  in  recently,  need  help?  \nproactive_outreach  \nLow  \nAsk  if  assistance  needed,  share  support  options.  \n1099  \nWha

In [65]:
def ask_with_history(session_id: str, question: str):
    """
    Ask a question with multi-user chat history support.
    Each session_id has its own stored conversation.
    """
    response = conversational_rag_chain.invoke(
        {"input": question},
        config={"configurable": {"session_id": session_id}}
    )
    return response["answer"]

In [66]:
# User 1 asks questions
print("User1:", ask_with_history("user1", "Where is my recent order? Still not shipped?"))
print("User1:", ask_with_history("user1", "Do you have any discount running on headphones? "))

# User 2 starts a different session
print("User2:", ask_with_history("user2", "Are there EMI options available on laptops?"))

# Check history for user1
history1 = get_session_history("user1")
print("\nUser1 Chat History:")
for msg in history1.messages:
    print(f"{msg.type.upper()}: {msg.content}")

User1: I've checked the shipment status, and it's still pending. I'll provide an update, and if it's delayed, I'll escalate the issue.
User1: Yes, we have an ongoing offer on headphones. I can inform you about the details and confirm your interest, and then forward you to our sales team for further assistance.
User2: EMI options are available on laptops. Explain EMI plans, confirm interest, and escalate to sales.

User1 Chat History:
HUMAN: Where is my recent order? Still not shipped?
AI: I've checked the shipment status, and it's still pending. I'll provide an update, and if it's delayed, I'll escalate the issue.
HUMAN: Do you have any discount running on headphones? 
AI: Yes, we have an ongoing offer on headphones. I can inform you about the details and confirm your interest, and then forward you to our sales team for further assistance.
